In [3]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
import sys
from pathlib import Path

current_dir = Path(os.getcwd())
parent_dir = str(current_dir.parent)
sys.path.append(parent_dir)

from src.embeddings.embedding_service import get_embedding

In [ ]:
def init_index(index_name="origin-of-species-index"):
    """
    Inicializa e retorna o índice do Pinecone.
    """
    PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
    if not PINECONE_API_KEY:
        raise ValueError("A variável de ambiente PINECONE_API_KEY não está definida.")

    
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    index = pc.Index(index_name)
    return index

def retrieve_relevant_chunks(query: str, index, top_k: int = 5, namespace: str = None):
    """
    Recupera os chunks mais relevantes para a query fornecida, utilizando o Pinecone.
    
    Parâmetros:
      query: A consulta em linguagem natural.
      index: O objeto de índice Pinecone.
      top_k: Número de resultados a retornar.
      namespace: (Opcional) Namespace a ser usado no índice.
    
    Retorna:
      A resposta do método de query do Pinecone, contendo os vetores mais similares.
    """
    
    query_embedding = get_embedding(query)
    
    result = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace
    )
    return result

In [ ]:
index = init_index()
query = "Como Darwin explica a seleção natural no livro?"
result = retrieve_relevant_chunks(query, index, top_k=5)
print("Resultados da recuperação:")
for match in result.matches:
    print(f"ID: {match.id}, Score: {match.score:.4f}")
    print(f"Texto: {match.metadata.get('text')[:200]}...")
    print("-----")

Resultados da recuperação:
ID: 0, Score: 0.8069
Texto: The Project Gutenberg eBook of On the Origin of Species By Means of Natural Selection This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and...
-----
ID: 6, Score: 0.7975
Texto: frequently recurring struggle for existence, it follows that any being, if it vary however slightly in any manner profitable to itself, under the complex and sometimes varying conditions of life, will...
-----
ID: 315, Score: 0.7922
Texto: selection may be extended. Effects of its adoption on the study of Natural history. Concluding remarks. As this whole volume is one long argument, it may be convenient to the reader to have the leadin...
-----
ID: 142, Score: 0.7885
Texto: principle of natural selection. If we admire the several ingenious contrivances, by which the flowers of the orchis and of many other plants are fertilised through insect agency, can we consider as eq...
-----
ID: 90, Score: 0.